In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [3]:
def l1_norm(u, u_ref):
    diff = numpy.sum(numpy.abs(u - u_ref))
    return diff

In [5]:
nx, ny = 41, 41
L = 1.0
xmin, xmax = 0.0, 1.0
ymin, ymax = 0.0, 1.0
Lx = (xmax - xmin)
Ly = (ymax - ymin)
dx = L / (nx - 1)
dy = L / (ny - 1)

x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)

In [ ]:
def stokes_2d_jacobi(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    p = p0.copy()
    conv = []
    diff = rtol + 1.0
    ite = 0
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        p[1:-1, 1:-1] = (((pn[1:-1, :-2] + pn[1:-1, 2:]) * dy**2 +
                          (pn[:-2, 1:-1] + pn[2:, 1:-1]) * dx**2 -
                          b[1:-1, 1:-1] * dx**2 * dy**2) /
                         (2.0 * (dx**2 + dy**2)))
        diff = l2_norm(p, pn)
        conv.append(diff)
        ite += 1
    return p, ite, conv